In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os
from azure.ai.projects import AIProjectClient
from azure.core.credentials import AzureKeyCredential 
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams


In [ ]:


load_dotenv(override=True) # take environment variables from .env.


project_endpoint = os.environ["PROJECT_ENDPOINT"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
openai_key = os.getenv("AZURE_OPENAI_KEY")
managed_identity_client_id = os.getenv("MANAGED_IDENTITY_CLIENT_ID")
new_credential = AzureKeyCredential(openai_key)
credential = DefaultAzureCredential(managed_identity_client_id=managed_identity_client_id)
# token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "test-agent")

In [14]:
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

list(project_client.agents.list_agents())

[{'id': 'asst_VsfVa3hCzB0ITiR1w7MQ9mMk', 'object': 'assistant', 'created_at': 1754185348, 'name': 'test-agent', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'},
 {'id': 'asst_3PbP9kqQZpAiRvoSeEiWq4iT', 'object': 'assistant', 'created_at': 1754185188, 'name': 'test-agent', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 

In [15]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)

print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'test-agent' created or updated successfully


In [16]:
agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=new_credential)

thread = project_client.agents.threads.create()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
        You must refer to references by id number
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.messages.list(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages = list(messages)
    messages.reverse()
    retrieval_result = agent_client.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]) for msg in messages if msg["role"] != "system"],
            target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text


functions = FunctionTool({ agentic_retrieval })
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset)

In [17]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value

print("Agent response:", output.replace(".", "\n"))

Agent response: Suburban belts display larger December brightening than urban cores despite the absolute light levels being higher downtown mainly because of holiday light displays and other festive activities during Christmas and New Year's
 The increase in holiday lighting, such as decorations and residential displays, tends to be more pronounced in suburban areas relative to their normal baseline lighting compared to already brightly lit urban centers where the relative increase is less dramatic
 This results in greater relative brightening in suburban belts during December even though the urban core has higher overall light levels [ref_id=0]


Regarding the sharp visibility of the Phoenix nighttime street grid from space, this is due to the city's regular grid layout of streets and blocks, which are clearly illuminated by street lighting
 The grid is further emphasized by features such as major transportation corridors (e
g
, Grand Avenue) and brightly lit commercial properties con

In [18]:
import json

retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1383,
    "output_tokens": 577
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Why do suburban areas show greater December brightening in nighttime lights compared to urban cores despite higher absolute light levels downtown?"
    },
    "query_time": "2025-08-03T09:09:22.044Z",
    "count": 1,
    "elapsed_ms": 363
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Why is the Phoenix nighttime street grid so sharply visible from space, while large stretches of interstate highways between Midwestern cities appear comparatively dim?"
    },
    "query_time": "2025-08-03T09:09:22.235Z",
    "count": 2,
    "elapsed_ms": 190
  },
  {
    "id": 3,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 49170
  }
]
Retrieval results
[
  {
    "type"

In [19]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night? Use the retrieval tool to answer this question."
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value


print("Agent response:", output.replace(".", "\n"))

Agent response: To find lava at night, satellite instruments like the VIIRS Day/Night Band (DNB) can detect the glowing light emitted by hot lava flows on active volcanoes
 These glowing lava flows produce bright lights distinguishable from city lights in nighttime satellite images
 Sufficient moonlight helps enhance visibility
 For example, glowing lava from eruptions such as those at Mount Etna in Italy or Kilauea in Hawaii is clearly visible from space as bright spots or flowing red/orange areas in nighttime images
 Thermal and infrared imaging can also be combined with visible light data to detect hot lava and monitor volcanic activity effectively
 This capability not only assists scientific research but also aids public safety by providing real-time monitoring of eruptions [ref_id=0, 1, 2, 3, 4]

